# **Auto encoder**

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from IPython import display
display.set_matplotlib_formats('svg')

<ipython-input-1-683616cd98db>:8: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  display.set_matplotlib_formats('svg')


 **Import & Processing the data**

In [2]:
data = np.loadtxt(open('sample_data/mnist_train_small.csv','rb'),delimiter=',')
labels = data[:,0]
data = data[:,1:]
dataNorm = data / np.max(data)
dataT = torch.tensor( dataNorm ).float()

**Create the DL Model**

In [3]:
def createTheMNISTAE():
  class aenet(nn.Module):
    def __init__(self):
      super().__init__()
      self.input = nn.Linear(784,150)
      self.enc = nn.Linear(150,15)
      self.lat = nn.Linear(15,150)
      self.dec = nn.Linear(150,784)
    def forward(self,x):
      x = F.relu( self.input(x) )
      codex = F.relu( self.enc(x) )
      x = F.relu( self.lat(codex) )
      y = torch.sigmoid( self.dec(x) )
      return y,codex
  net = aenet()
  lossfun = nn.MSELoss()
  optimizer = torch.optim.Adam(net.parameters(),lr=.001) #change lr here
  return net,lossfun,optimizer
net,lossfun,optimizer = createTheMNISTAE()
X = dataT[:5,:]
yHat = net(X)
print('Input shape:')
print(X.shape)
print(' ')
print(type(yHat),len(yHat))
print(' ')
print('Shape of model output:')
print(yHat[0].shape)
print(' ')
print('Shape of encoding layer output:')
print(yHat[1].shape)

Input shape:
torch.Size([5, 784])
 
<class 'tuple'> 2
 
Shape of model output:
torch.Size([5, 784])
 
Shape of encoding layer output:
torch.Size([5, 50])



To change the activation function, you can modify the forward method of the aenet class in the createTheMNISTAE function. Here's an example of using the hyperbolic tangent (tanh) activation function instead of the rectified linear unit (ReLU) function:
```
def createTheMNISTAE():
    class aenet(nn.Module):
        def __init__(self):
            super().__init__()
            self.input = nn.Linear(784, 150)
            self.enc = nn.Linear(150, 15)
            self.lat = nn.Linear(15, 150)
            self.dec = nn.Linear(150, 784)
        def forward(self, x):
            x = torch.tanh(self.input(x))
            codex = torch.tanh(self.enc(x))
            x = torch.tanh(self.lat(codex))
            y = torch.sigmoid(self.dec(x))
            return y, codex
    
    net = aenet()
    lossfun = nn.MSELoss()
    optimizer = torch.optim.Adam(net.parameters(), lr=.001)
    return net, lossfun, optimizer

net, lossfun, optimizer = createTheMNISTAE()
X = dataT[:5, :]
yHat = net(X)
print('Input shape:')
print(X.shape)
print(' ')
print(type(yHat), len(yHat))
print(' ')
print('Shape of model output:')
print(yHat[0].shape)
print(' ')
print('Shape of encoding layer output:')
print(yHat[1].shape)

```





```
class aenet(nn.Module):
    def __init__(self):
        super().__init__()
        self.input = nn.Linear(784, 300)
        self.enc = nn.Linear(300, 50)
        self.lat = nn.Linear(50, 300)
        self.dec = nn.Linear(300, 784)
        self.output = nn.Linear(784, 784)
        
    def forward(self, x):
        x = F.relu(self.input(x))
        codex = F.relu(self.enc(x))
        x = F.relu(self.lat(codex))
        y = torch.sigmoid(self.dec(x))
        z = torch.sigmoid(self.output(y))
        return z, codex


In this modification, we added an additional linear layer called self.output with input and output dimensions of 784, the same as the input and output dimensions of the autoencoder. We also changed the input and output dimensions of the first and last linear layers to match the input and output dimensions of self.output. Finally, we modified the forward method to include the additional layer and return its output as well.
```



In [4]:
def function2trainTheModel():
  numepochs = 10000 #change here
  net,lossfun,optimizer = createTheMNISTAE()
  losses = torch.zeros(numepochs)
  for epochi in range(numepochs):
    randomidx = np.random.choice(dataT.shape[0],size=32)
    X = dataT[randomidx,:]
    yHat = net(X)[0] 
    loss = lossfun(yHat,X)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    losses[epochi] = loss.item()
  return losses,net

In [6]:
losses,net = function2trainTheModel()
print(f'Final loss: {losses[-1]:.4f}')
plt.plot(losses,'.-')
plt.xlabel('Epochs')
plt.ylabel('Model loss')
plt.show()

KeyboardInterrupt: ignored

In [ ]:
yHat,latent = net(dataT)
print(yHat.shape)
print(latent.shape)
fig,ax = plt.subplots(1,2,figsize=(15,5))
ax[0].hist(latent.flatten().detach(),100)
ax[0].set_xlabel('Latent activation value')
ax[0].set_ylabel('Count')
ax[0].set_title('Distribution of latent units activations')
ax[1].imshow(latent.detach(),aspect='auto',vmin=0,vmax=10)
ax[1].set_xlabel('Latent node')
ax[1].set_ylabel('Image number')
ax[1].set_title('All latent activations')
plt.show()

In [ ]:
sourcecode = np.zeros((latent.shape[1],10))
for i in range(10):
  digidx = np.where(labels==i)
  sourcecode[:,i] = torch.mean(latent[digidx,:],axis=1).detach()
fig = plt.figure(figsize=(8,5))
plt.plot(sourcecode,'s-')
plt.legend(range(10),loc=(1.01,.4))
plt.xticks(range(15))
plt.xlabel('Latent node number')
plt.ylabel('Activation')
plt.title("The model's internal representation of the numbers")
plt.show()

In [ ]:
pcaData = PCA(n_components=15).fit(data) 
pcaCode = PCA( ).fit(latent.detach())
plt.plot(100*pcaData.explained_variance_ratio_,'s-',label='Data PCA')
plt.plot(100*pcaCode.explained_variance_ratio_,'o-',label='Code PCA')
plt.xlabel('Components')
plt.ylabel('Percent variance explained')
plt.title('PCA scree plot')
plt.legend()
plt.show()
scoresData = pcaData.fit_transform(data)
scoresCode = pcaCode.fit_transform(latent.detach())
fig,ax = plt.subplots(1,2,figsize=(15,5))
for lab in range(10):
  ax[0].plot(scoresData[labels==lab,0],scoresData[labels==lab,1],'o',markersize=3,alpha=.4)
  ax[1].plot(scoresCode[labels==lab,0],scoresCode[labels==lab,1],'o',markersize=3,alpha=.4)
for i in range(2):
  ax[i].set_xlabel('PC1 projection')
  ax[i].set_ylabel('PC2 projection')
  ax[i].legend(range(10))
ax[0].set_title('PCA of data')
ax[1].set_title('PCA of latent code')
plt.show()
fig,ax = plt.subplots(1,3,figsize=(15,3))
ax[0].imshow(dataT[0,:].view(28,28),cmap='gray')
ax[1].plot(dataT[0,:],'ks')
ax[1].set_xlabel('Pixels (vectorized)')
ax[1].set_ylabel('Intensity value')
ax[2].plot(latent[0,:].detach(),'ks')
ax[2].set_xlabel('Latent units')
ax[2].set_ylabel('Activation (a.u.)')
plt.show()